In [ ]:
# python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7gripper'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

DAT_ROOT = "stowing-deep"
RES_ROOT = "stowing-deep-result"

TEST_DATA_PATH = os.path.join(DATA_PATH, DAT_ROOT)
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, RES_ROOT)
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

# obj1

In [3]:
RTYPE = "indy7gripper"
DAT_DIR = '20210826-234456'
FILE_OPTION = "obj_1"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_1"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
GRASP_SAMPLE = 100
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100


### test

In [3]:
I_START = 0
I_END = 10
N_DAT = I_END-I_START

In [4]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 7.5     7.0     8.0     7.5     -------
========== 1 / 10 (eta: 30.0 / 300.2 s) =========
------- 9.2     8.0     7.1     7.1     -------
========== 2 / 10 (eta: 61.4 / 306.8 s) =========
------- 6.7     7.7     7.2     7.6     -------
========== 3 / 10 (eta: 90.5 / 301.7 s) =========
------- 23.8    25.9    10.1    7.7     -------
========== 4 / 10 (eta: 158.0 / 394.9 s) =========
------- 10.4    8.1     7.2     20.2    -------
========== 5 / 10 (eta: 203.9 / 407.7 s) =========
------- 7.5     8.3     6.9     7.1     -------
========== 6 / 10 (eta: 233.7 / 389.5 s) =========
------- 14.8    6.4     7.3     7.3     -------
========== 7 / 10 (eta: 269.6 / 385.1 s) =========
------- 16.7    6.7     7.3     7.8     -------
========== 8 / 10 (eta: 308.1 / 385.1 s) =========
------- 6.3     6.7     6.9     7.4     -------
========== 9 / 10 (eta: 335.5 / 372.7 s) =========
------- 15.1    7.2     7.2     7.3     -------
========== 10 / 10 (eta: 372.2 / 372.2 s) =========


# obj3

In [6]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
FILE_OPTION = "obj_3"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [7]:
I_START=0
N_DAT = 50-I_START

### test

In [8]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 16.2    5.0     6.7     6.1     -------
========== 1 / 50 (eta: 34.0 / 1699.7 s) =========
------- 6.8     19.3    8.1     6.5     -------
========== 2 / 50 (eta: 74.7 / 1867.7 s) =========
------- 99.8    12.4    22.8    6.6     -------
========== 3 / 50 (eta: 216.3 / 3605.6 s) =========
------- 104.8   25.2    8.0     55.8    -------
========== 4 / 50 (eta: 410.2 / 5127.2 s) =========
------- 91.9    6.9     6.7     5.9     -------
========== 5 / 50 (eta: 521.6 / 5215.6 s) =========
------- 34.3    5.1     7.2     8.0     -------
========== 6 / 50 (eta: 576.2 / 4801.3 s) =========
------- 7.5     7.1     6.0     6.3     -------
========== 7 / 50 (eta: 603.0 / 4307.1 s) =========
------- 22.0    6.3     5.9     5.8     -------
========== 8 / 50 (eta: 643.1 / 4019.2 s) =========
------- 67.5    5.5     5.9     5.6     -------
========== 9 / 50 (eta: 727.6 / 4042.3 s) =========
------- 20.2    23.7    5.8     8.6     -------
========== 10 / 50 (eta: 785.9 / 3929.5 s) =========
-

# obj3_pole

In [4]:
RTYPE = "indy7gripper"
DAT_DIR = '20210707-034434'
# DAT_DIR = '20210702-152658'
FILE_OPTION = "obj_3_pole"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

# CNAME_LIST = ['None','Tool','ToolReach','Full'] #,'Pairwise']
# GRASP_SAMPLE = 100
# VISUALIZE = True
# PLAY_RESULT = True
# TIMEOUT_MOTION = 5
# MAX_TIME = 100
# SHOW_STATE = False


In [5]:
I_START=0
N_DAT = 50-I_START

### test

In [6]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

OSError: [Errno 2] No such file or directory: '/home/rnb/Projects/rnb-planning/data/stowing-deep/indy7gripper/20210707-034434'

# obj3_hard

In [7]:
RTYPE = "indy7gripper"
DAT_DIR = '20210826-234456'
# DAT_DIR = '20210702-152658'
FILE_OPTION = "obj_3_hard"
# DAT_DIR, FILE_OPTION = "20210613-023004", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-114040", "obj_1"
# DAT_DIR, FILE_OPTION = "20210612-151342", "obj_3"
# DAT_DIR, FILE_OPTION = "20210612-161536", "obj_3_pole"

# RTYPE = "panda"
# DAT_DIR = '20210705-182244'
# FILE_OPTION = "obj_3_pole"
# # DAT_DIR, FILE_OPTION = "20210613-022832", "obj_1"
# # # DAT_DIR, FILE_OPTION = "20210612-221934", "obj_1"

CNAME_LIST = ['ToolReach','Full'] #,'Pairwise']
# CNAME_LIST = ['ToolReach'] #,'Pairwise']
GRASP_SAMPLE = 100
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100

In [8]:
I_START=0
I_END = 100
N_DAT = I_END-I_START

### test

In [9]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')
gtimer.tic("full_loop")
for fid in range(N_DAT):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid+I_START),
                              '--cname', cname,
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid+1, N_DAT), 1)))

------- 107.5   103.0   -------
========== 1 / 100 (eta: 210.4 / 21042.2 s) =========
------- 4.5     25.3    -------
========== 2 / 100 (eta: 240.3 / 12013.9 s) =========
------- 103.0   3.8     -------
========== 3 / 100 (eta: 347.0 / 11567.3 s) =========
------- 103.2   105.8   -------
========== 4 / 100 (eta: 556.0 / 13900.6 s) =========
------- 92.2    31.4    -------
========== 5 / 100 (eta: 679.6 / 13592.5 s) =========
------- 4.0     8.9     -------
========== 6 / 100 (eta: 692.6 / 11542.6 s) =========
------- 24.4    3.6     -------
========== 7 / 100 (eta: 720.6 / 10294.5 s) =========
------- 103.0   113.0   -------
========== 8 / 100 (eta: 936.5 / 11706.6 s) =========
------- 105.5   103.3   -------
========== 9 / 100 (eta: 1145.3 / 12725.5 s) =========
------- 102.7   103.1   -------
========== 10 / 100 (eta: 1351.1 / 13511.2 s) =========
------- 104.8   103.3   -------
========== 11 / 100 (eta: 1559.2 / 14174.9 s) =========
------- 102.8   107.5   -------
========== 12 / 1

KeyboardInterrupt: 